In [1]:
include("../src/Utero.jl")
using .Utero

In [2]:
@show x = Utero.Tracker(7.0)
Utero.ForwardBackward(x -> 2.0*x + x, x)

x = Utero.Tracker(7.0) = Main.Utero.Tracker{Float64}(7.0, 26.53221675450974, Any[], Any[])
0.0
2.0
7.0


Dict{Any, Any} with 3 entries:
  49.0518 => 1.0
  58.8504 => 7.0
  26.5322 => 2.0

In [3]:
Utero.⬅ctx.Tape

3-element Vector{Any}:
 Main.Utero.Tracker{Float64}(2.0, 58.850423331004606, Any[Main.Utero.var"#11#12"{Main.Utero.var"#MultiplicationLinker#33"{Float64, Float64}, Int64}(Main.Utero.var"#MultiplicationLinker#33"{Float64, Float64}(2.0, 7.0), 1)], Any[49.051807201774224])
 Main.Utero.Tracker{Float64}(7.0, 26.53221675450974, Any[Main.Utero.var"#11#12"{Main.Utero.var"#MultiplicationLinker#33"{Float64, Float64}, Int64}(Main.Utero.var"#MultiplicationLinker#33"{Float64, Float64}(2.0, 7.0), 2)], Any[49.051807201774224])
 Main.Utero.Tracker{Float64}(14.0, 49.051807201774224, Any[], Any[])

In [4]:
# should be 42
Utero.ForwardBackward(x -> x^2.0 + x^2.0, Utero.Tracker(3.2)) 

0.0
1.0
1.0
11.910664292410175
12.8
11.910664292410175
12.8


Dict{Any, Any} with 6 entries:
  95.7721 => 1.0
  19.6345 => 1.0
  95.151  => 1.0
  4.39573 => 25.6
  81.0008 => 11.9107
  95.6457 => 11.9107

In [5]:
# Gradient should be 56.21
Utero.ForwardBackward(w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), Utero.Tracker(7.0))

0.0
1.0
1.0
0.3005925437436371
28.661377998453734


KeyError: KeyError: key 93.02539614317567 not found

# Differentiation for Scalar Parameters

In [6]:
x = 12 # treated as constant 

Params = IdDict()
w = Utero.Tracked(7.0, Params)
grads = Utero.GetGradient(_ -> 2 * (((2*w + x) - w) * w) + sin(w^2), Params) 
# should be approx. 56.21
println(grads)

Params = IdDict()
w = Utero.Tracked(25.0, Params)
grads = Utero.GetGradient(_ -> sin(w)^2, Params)
# should be approx. -0.26
println(grads)

Params = IdDict()
w1 = Utero.Tracked(13.2, Params); w2 = Utero.Tracked(89.0, Params)
grads = Utero.GetGradient(_ -> (cos(w2)^3 * 4sin(w1))^2 + cos(w1*w2)^2, Params) 
# should be approx. (27.53, 3.0428)
println(grads)


UndefVarError: UndefVarError: `Tracked` not defined

# Differentiation for Tensor Parameters

In [7]:

Parameters = IdDict()
w1 = Utero.Tracked(rand(2, 2), Parameters)
w2 = Utero.Tracked(rand(2, 2), Parameters)
w3 = Utero.Tracked(rand(2, 2), Parameters)

grads = Utero.GetGradient(x -> prod(*(w3, *(w2, *(w1, x)))), rand(2,1), Parameters)

UndefVarError: UndefVarError: `Tracked` not defined

The product of the parameter matrices should match A.

In [8]:
A = rand(2,2)
X = rand(100, 2)
Y = mapslices(v -> (A * v), X, dims=(2))


println(A)
println((w1 * w2 * w3).val)

for (x, y) in zip(eachrow(X), eachrow(Y))
    x = vec(x); y = vec(y)
    grads = Utero.GetGradient(x -> Utero.mse(*(w3, *(w2, *(w1, x))), y), x, Parameters)
    Utero.GradientStep(0.01, Parameters, grads)
end

println("Now, they should match")
println(A)
println((w1 * w2 * w3).val)

[0.9388171203553851 0.3286512699972812; 0.7750698881927134 0.4335952028578305]


UndefVarError: UndefVarError: `w1` not defined